# Spam Ham Classification

In [1]:
import pandas as pd

In [2]:
"""
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

"""

"\nnltk.download('punkt')\nnltk.download('averaged_perceptron_tagger')\nnltk.download('stopwords')\nnltk.download('wordnet')\n\n"

In [3]:
import nltk


In [4]:
df=pd.read_csv('spamdata.csv')

In [5]:
df.head(10)

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


## Pre-processing

In [6]:
import string

In [7]:
punctuations=string.punctuation

In [8]:
from nltk.corpus import stopwords

In [9]:
stop_word_list=stopwords.words('English')

In [10]:
from nltk.stem.wordnet import WordNetLemmatizer

In [11]:
lem= WordNetLemmatizer()

In [12]:
def _cleandata(text):
    cleaned_text=text.lower()
    cleaned_text="".join(c for c in cleaned_text if c not in punctuations)
    words=cleaned_text.split()
    words=[w for w in words if w not in stop_word_list]
    
    words=[lem.lemmatize(word,"v") for word in words]
    words=[lem.lemmatize(word,"n") for word in words]
    
    cleaned_text=" ".join(words)
    
    
    return cleaned_text

In [13]:
_cleandata('I will be playing a game today.')

'play game today'

In [14]:
 df['clean_text']=df['text'].apply(_cleandata)

In [15]:
df.head(10)

,label,text,clean_text
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think go usf live around though
5,spam,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey darling 3 week word back id like f...
6,ham,Even my brother is not like to speak with me. ...,even brother like speak treat like aid patent
7,ham,As per your request 'Melle Melle (Oru Minnamin...,per request melle melle oru minnaminunginte nu...
8,spam,WINNER!! As a valued network customer you have...,winner value network customer select receivea ...
9,spam,Had your mobile 11 months or more? U R entitle...,mobile 11 month u r entitle update latest colo...


In [16]:
# Did not remove numbers, support words/domain specific words, slangs, typos.

## Feature Engineering

In [17]:
#meta features

In [18]:
df['word_count']=df['text'].apply(lambda x: len(x.split()))

In [19]:
df['word_count_clean']=df['clean_text'].apply(lambda x: len(x.split()))

In [20]:
df['num_digits']=df['text'].apply(lambda x: sum([1 if w.isdigit() else 0 for w in x.split()]))

In [21]:
df.head(10)

,label,text,clean_text,word_count,word_count_clean,num_digits
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,20,16,0
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni,6,6,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,28,23,2
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say,11,9,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think go usf live around though,13,8,0
5,spam,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey darling 3 week word back id like f...,32,19,1
6,ham,Even my brother is not like to speak with me. ...,even brother like speak treat like aid patent,16,8,0
7,ham,As per your request 'Melle Melle (Oru Minnamin...,per request melle melle oru minnaminunginte nu...,26,16,0
8,spam,WINNER!! As a valued network customer you have...,winner value network customer select receivea ...,26,18,1
9,spam,Had your mobile 11 months or more? U R entitle...,mobile 11 month u r entitle update latest colo...,29,18,2


In [22]:
pos_dict={'noun': ['NNP','NN','NNS','NNPS'],'verb':['VB','VBZ','VBD','VBN','VBG']}

In [23]:
#NN 	noun, singular or mass 	tiger, chair, laughter
#NNS 	noun, plural 	tigers, chairs, insects
#NNP 	noun, proper singular 	Germany, God, Alice
#NNPS 	noun, proper plural 	we met two Christmases ago 
#VB 	verb, base form 	think
#VBZ 	verb, 3rd person singular present 	she thinks
#VBP 	verb, non-3rd person singular present 	I think
#VBD 	verb, past tense 	they thought
#VBN 	verb, past participle 	a sunken ship
#VBG 	verb, gerund or present participle 	thinking is fun 

In [24]:
def pos_check(txt,family):
    tags=nltk.pos_tag(nltk.word_tokenize(txt))
    count=0
    for tag in tags:
        tag=tag[1]
        if tag in pos_dict[family]:
            count+=1
    return count       

In [25]:
pos_check("They are playing in the ground with all other kids",'verb')

1

In [26]:
df['pos_noun_count']=df['text'].apply(lambda x: pos_check(x,'noun'))
df['pos_verb_count']=df['text'].apply(lambda x: pos_check(x,'verb'))

In [27]:
df.head()

,label,text,clean_text,word_count,word_count_clean,num_digits,pos_noun_count,pos_verb_count
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,20,16,0,10,1
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni,6,6,0,4,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,28,23,2,13,3
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say,11,9,0,3,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think go usf live around though,13,8,0,1,4


In [28]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [29]:
cv=CountVectorizer()

In [30]:
cv.fit(df['clean_text'].values)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [31]:
count_vectors=cv.transform(df['clean_text'].values)

In [32]:
tfidf=TfidfVectorizer()
tfidf.fit(df['clean_text'].values)
tfidf_vectors=tfidf.transform(df['clean_text'].values)

In [33]:
ngram_tfidf=TfidfVectorizer(max_features=1000,ngram_range=(2,4))
ngram_tfidf.fit(df['clean_text'].values)
ngram_tfidf_vectors=ngram_tfidf.transform(df['clean_text'].values)

In [34]:
tfidf_dict_1000=dict(zip(tfidf.get_feature_names(),tfidf.idf_))

In [35]:
tfidf_dict_1000

{'008704050406': 8.527076498901426,
 '0089my': 8.93254160700959,
 '0121': 8.93254160700959,
 '01223585236': 8.93254160700959,
 '01223585334': 8.527076498901426,
 '0125698789': 8.93254160700959,
 '02': 8.93254160700959,
 '020603': 8.016250875135436,
 '0207': 8.527076498901426,
 '02070836089': 8.93254160700959,
 '02072069400': 8.93254160700959,
 '02073162414': 8.527076498901426,
 '02085076972': 8.93254160700959,
 '020903': 8.527076498901426,
 '021': 8.527076498901426,
 '050703': 8.527076498901426,
 '0578': 8.527076498901426,
 '06': 8.93254160700959,
 '060505': 8.93254160700959,
 '061104': 8.93254160700959,
 '07008009200': 8.93254160700959,
 '07046744435': 8.93254160700959,
 '07090201529': 8.93254160700959,
 '07090298926': 8.93254160700959,
 '07099833605': 8.93254160700959,
 '071104': 8.527076498901426,
 '07123456789': 8.527076498901426,
 '0721072': 8.93254160700959,
 '07732584351': 8.93254160700959,
 '07734396839': 8.527076498901426,
 '07742676969': 8.527076498901426,
 '07753741225': 8.9

In [36]:
tfdf_1000=pd.DataFrame(list(tfidf_dict_1000.items()), columns=['word', 'tfidf_value'])

In [37]:
tfdf_1000

,word,tfidf_value
0,008704050406,8.527076
1,0089my,8.932542
2,0121,8.932542
3,01223585236,8.932542
4,01223585334,8.527076
5,0125698789,8.932542
6,02,8.932542
7,020603,8.016251
8,0207,8.527076
9,02070836089,8.932542


In [38]:
tfdf_1000=pd.DataFrame(list(tfidf_dict_1000.items()), columns=['word', 'tfidf_value'])

In [39]:
tfdf_1000

,word,tfidf_value
0,008704050406,8.527076
1,0089my,8.932542
2,0121,8.932542
3,01223585236,8.932542
4,01223585334,8.527076
5,0125698789,8.932542
6,02,8.932542
7,020603,8.016251
8,0207,8.527076
9,02070836089,8.932542


In [40]:
from scipy.sparse import hstack,csr_matrix

In [41]:
df.columns

Index(['label', 'text', 'clean_text', 'word_count', 'word_count_clean',
       'num_digits', 'pos_noun_count', 'pos_verb_count'],
      dtype='object')

In [42]:
meta_features=['word_count', 'word_count_clean',
       'num_digits', 'pos_noun_count', 'pos_verb_count']

In [43]:
feature_set1=df[meta_features]

In [44]:
train=hstack([tfidf_vectors,csr_matrix(feature_set1)],"csr")

In [45]:
train

<5572x8212 sparse matrix of type '<class 'numpy.float64'>'
	with 69090 stored elements in Compressed Sparse Row format>

## Machine Learning based Predictions 

In [46]:
from sklearn.preprocessing import LabelEncoder

In [47]:
target=df['label'].values

In [48]:
target=LabelEncoder().fit_transform(target)

In [49]:
target

array([0, 0, 1, ..., 0, 0, 0])

In [50]:
from sklearn.model_selection import train_test_split

In [51]:
train_x,val_x,train_y,val_y=train_test_split(train,target)

In [52]:
from sklearn import naive_bayes
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import ensemble
from sklearn.metrics import accuracy_score

In [53]:
model_nv= naive_bayes.MultinomialNB()
model_nv.fit(train_x,train_y)
preds=model_nv.predict(val_x)
accuracy_score(preds,val_y )

0.8916008614501076

In [54]:
model_lr= LogisticRegression()
model_lr.fit(train_x,train_y)
preds_lr=model_lr.predict(val_x)
accuracy_score(preds_lr,val_y )

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9540559942569993

In [55]:
model_svm= svm.SVC()
model_svm.fit(train_x,train_y)
preds_svm=model_svm.predict(val_x)
accuracy_score(preds_svm,val_y )

/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.8578607322325915

In [56]:
model_extree= ensemble.ExtraTreesClassifier()
model_extree.fit(train_x,train_y)
preds_extree=model_extree.predict(val_x)
accuracy_score(preds_extree,val_y )

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9720028715003589

## Deep Learning based predictions 

In [57]:
import numpy as np

In [58]:
import keras

Using TensorFlow backend.


In [59]:
# To get embeddings corresponding to every word in the pretained vector(glove file)

In [60]:
embedding_index={}

In [61]:
for i,line in enumerate(open("glove.6B.100d.txt")):
    value=line.split()
    #print(line)
    embedding_index[value[0]]=np.array(value[1:],dtype='float32')

In [62]:
len(embedding_index.get('the'))

100

In [63]:
# To convert text data into these word embeddings representation

In [64]:
from keras.preprocessing import text,sequence

In [65]:
token=text.Tokenizer()
token.fit_on_texts(df['text'])

In [66]:
word_index=token.word_index

In [67]:
word_index

{'i': 1,
 'to': 2,
 'you': 3,
 'a': 4,
 'the': 5,
 'u': 6,
 'and': 7,
 'in': 8,
 'is': 9,
 'me': 10,
 'my': 11,
 'for': 12,
 'your': 13,
 'it': 14,
 'of': 15,
 'call': 16,
 'have': 17,
 'on': 18,
 '2': 19,
 'that': 20,
 'now': 21,
 'are': 22,
 'so': 23,
 'but': 24,
 'not': 25,
 'or': 26,
 'do': 27,
 'can': 28,
 'at': 29,
 "i'm": 30,
 'get': 31,
 'be': 32,
 'will': 33,
 'if': 34,
 'ur': 35,
 'with': 36,
 'just': 37,
 'no': 38,
 'we': 39,
 'this': 40,
 'gt': 41,
 '4': 42,
 'lt': 43,
 'up': 44,
 'when': 45,
 'ok': 46,
 'free': 47,
 'from': 48,
 'how': 49,
 'go': 50,
 'all': 51,
 'out': 52,
 'what': 53,
 'know': 54,
 'like': 55,
 'good': 56,
 'then': 57,
 'got': 58,
 'was': 59,
 'come': 60,
 'its': 61,
 'am': 62,
 'time': 63,
 'only': 64,
 'day': 65,
 'love': 66,
 'there': 67,
 'send': 68,
 'he': 69,
 'want': 70,
 'text': 71,
 'as': 72,
 'txt': 73,
 'one': 74,
 'going': 75,
 'by': 76,
 'home': 77,
 "i'll": 78,
 'need': 79,
 'about': 80,
 'r': 81,
 'lor': 82,
 'sorry': 83,
 'stop': 84,
 'st

In [68]:
#To convert text into sequence of tokens and pad them to ensure equal length vectors are obtained

In [69]:
trainx,valx,trainy,valy=train_test_split(df['text'],target)

In [70]:
trainx=sequence.pad_sequences(token.texts_to_sequences(trainx),maxlen=70)
valx=sequence.pad_sequences(token.texts_to_sequences(valx),maxlen=70)


In [71]:
embedding_matrix=np.zeros(((len(word_index)+1),100)) #100=>length of embedding index

In [72]:
for word,i in word_index.items():
    embedding_vector=embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector

In [73]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.046539  ,  0.61966002,  0.56647003, ..., -0.37616   ,
        -0.032502  ,  0.80620003],
       [-0.18970001,  0.050024  ,  0.19084001, ..., -0.39804   ,
         0.47646999, -0.15983   ],
       ...,
       [-0.12201   ,  0.20465   ,  0.53192002, ..., -0.164     ,
        -0.41637   ,  0.66426003],
       [-0.41600999,  0.16974001,  0.43457001, ..., -0.083432  ,
         0.49520999,  0.77258003],
       [ 0.46118999,  0.3567    ,  0.026078  , ..., -0.0056525 ,
         0.55176002,  0.77563   ]])

In [74]:
def train_model(classifier,feature_vector_train,label,feature_vector_val,validy):
    classifier.fit(feature_vector_train,label)
    predictions=classifier.predict(feature_vector_val)
    predictions=predictions.argmax(axis=-1)
    return accuracy_score(predictions,validy)

In [75]:
from keras import layers, models,optimizers

In [76]:
def create_cnn():
    input_layer=layers.Input((70,))
    
    # Length of total vocabulary, 100 as the length of word_vector, since we are using pretrained vects we will pass those weights which are present in our embedding matrix
    embedding_layer=layers.Embedding(len(word_index)+1,100,weights=[embedding_matrix],trainable=False)(input_layer)
    
    #number of convolution units, window size,
    conv_layer=layers.Convolution1D(100,3,activation="relu")(embedding_layer)
    
    #pass output of conv layer and extract most mp features
    pooling_layer=layers.GlobalMaxPool1D()(conv_layer)
    
    #output dense layer 50 neuron => neural network for classification 
    output_layer=layers.Dense(50,activation='relu')(pooling_layer)
    #dropout layer used for regularization in neural network=> specify the number the units you want to dropout randomly(25%)
    output_layer=layers.Dropout(0.25)(output_layer)
    
    #final actual classification-sigmoid to perform binary classification
    output_layer=layers.Dense(1,activation='sigmoid')(output_layer)
    
    model=models.Model(inputs=input_layer,outputs=output_layer)
    model.compile(optimizer=optimizers.Adam(),loss="binary_crossentropy")
    
    return model
    
    
    
    
    

In [2]:
classifier=create_cnn()
accuracy=train_model(classifier,trainx,trainy,valx,valy,is_neural_net=True)

NameError: name 'create_cnn' is not defined

In [3]:
print ("CNN, Word Embeddings",  accuracy)

NameError: name 'accuracy' is not defined